In [3]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/cifar10-python/cifar-10-python.tar.gz
/kaggle/input/cifar10-python/cifar-10-batches-py/data_batch_1
/kaggle/input/cifar10-python/cifar-10-batches-py/data_batch_2
/kaggle/input/cifar10-python/cifar-10-batches-py/batches.meta
/kaggle/input/cifar10-python/cifar-10-batches-py/test_batch
/kaggle/input/cifar10-python/cifar-10-batches-py/data_batch_3
/kaggle/input/cifar10-python/cifar-10-batches-py/data_batch_5
/kaggle/input/cifar10-python/cifar-10-batches-py/data_batch_4
/kaggle/input/cifar10-python/cifar-10-batches-py/readme.html


In [4]:
import torch.nn as nn
import torch 
import torch.nn.functional as F 

In [5]:
from torch.utils.data import DataLoader, TensorDataset

def unpickle(file):
    import pickle
    with open(file, 'rb') as fo:
        dict = pickle.load(fo, encoding='bytes')
    return dict


train_batch = {}
test_batch = unpickle('/kaggle/input/cifar10-python/cifar-10-batches-py/test_batch')
for i in range(5):
    train_batch[f'batch_{i+1}'] = unpickle("/kaggle/input/cifar10-python/cifar-10-batches-py/data_batch_%d"%(i + 1))


def get_dataset(batch_dict,test_dict):


    training_image = torch.tensor(torch.cat((torch.tensor(batch_dict['batch_1'][b'data']),torch.tensor(batch_dict['batch_2'][b'data']),torch.tensor(batch_dict['batch_3'][b'data']),torch.tensor(batch_dict['batch_4'][b'data']),torch.tensor(batch_dict['batch_5'][b'data'])),0),dtype = torch.float32)
    training_labels = torch.tensor(torch.cat((torch.tensor(batch_dict['batch_1'][b'labels']),torch.tensor(batch_dict['batch_2'][b'labels']),torch.tensor(batch_dict['batch_3'][b'labels']),torch.tensor(batch_dict['batch_4'][b'labels']),torch.tensor(batch_dict['batch_5'][b'labels']))))     

    test_images = torch.tensor(test_dict[b'data'],dtype = torch.float32)
    test_labels = torch.tensor(test_dict[b'labels'])

    training_image = training_image.view(50000,3,32,32)
    test_images = test_images.view(test_images.shape[0],3,32,32)

    return training_image,training_labels,test_images,test_labels


def make_batches(batch_size=16):
    training_image,training_labels,test_images,test_labels = get_dataset(train_batch,test_batch)
    train_dataset = TensorDataset(training_image,training_labels)
    test_dataset = TensorDataset(test_images,test_labels)
    train_loader = DataLoader(train_dataset,shuffle=True,batch_size=batch_size)
    test_loader = DataLoader(test_dataset,shuffle =True,batch_size=batch_size)

    return train_loader,test_loader

In [6]:
import torch 
import torch.nn as nn
def training(model,criterion,optimizer,train_loader,epochs=10,device='cuda'):
    training_loss = 0.0
    training_accuracy = 0
    for epoch in range(epochs):
        for i,(images,labels) in enumerate(train_loader):
            images= images.to(device)
            labels = labels.to(device)
            output = model(images)

            loss = criterion(output,labels)

            # training_loss += loss.item()
            optimizer.zero_grad()

            loss.backward()

            optimizer.step()

            _, predicted = torch.max(output.data, 1)

            training_accuracy += (predicted == labels).sum().item()
            if ((i+1)%100 == 0):
                print(f'epoch:{epoch + 1},loss:{loss.item():.4f}')
        print(f'accuracy:{training_accuracy*100/50000}')
        training_accuracy = 0    

#     torch.save(model.state_dict(),'/kaggle/working/model.pth')

    return model


def evaluation(model,test_loader,device='cuda'):
    with torch.no_grad():
        n_correct = 0
        n_samples = 0
        test_loss = 0.0
        test_accuracy = 0
        for i,(images, labels) in enumerate(test_loader):
            images= images.to(device)
            labels = labels.to(device)
            outputs = model(images)
            _, predicted = torch.max(outputs.data, 1)
            n_samples += labels.size(0)
            n_correct += (predicted == labels).sum().item()
            
        acc = 100.0 * n_correct / n_samples
        
        print(f'Accuracy of the network on the 10000 test images: {acc} %')

In [13]:
class ResidualBlock(nn.Module):
    
    def __init__(self,input_channels,no_of_filters,kernel_size=3,stride1 = 1,stride2 = 1,padding = 1):
        
        super(ResidualBlock,self).__init__()
        
        self.resblock = nn.Sequential(nn.Conv2d(input_channels,no_of_filters,kernel_size = kernel_size,stride = stride1,padding = padding),
                                      nn.BatchNorm2d(no_of_filters),
                                      nn.ReLU(),
                                      nn.Conv2d(no_of_filters,no_of_filters,kernel_size = kernel_size,stride = stride2,padding = padding),
                                      nn.BatchNorm2d(no_of_filters),
                                      nn.ReLU()
                                     )
    def forward(self,x):
        
        x = self.resblock(x)
        return x 
    
    
class IdentityBlock(nn.Module):
    
        def __init__(self,input_channels,no_of_filters,kernel_size=3,stride = 1 ,padding = 0):
            
            super(IdentityBlock,self).__init__()
            
            self.idblock = nn.Sequential(nn.Conv2d(input_channels,no_of_filters,kernel_size = kernel_size,stride = stride,padding = padding),
                                          nn.BatchNorm2d(no_of_filters),
                                          nn.ReLU()
                                         )
        def forward(self,x):
        
            x = self.idblock(x)
            return x 

In [17]:
class Resnet32(nn.Module):
    
    def __init__(self):
        super(Resnet32,self).__init__()
        
        self.block1_0 = nn.Conv2d(3,64,3,padding = 1) 
        
        self.block1_1 = ResidualBlock(64,64,kernel_size = 3)                                             
        self.block1_2 = ResidualBlock(64,64,kernel_size = 3)
        self.block1_3 = ResidualBlock(64,64,kernel_size = 3)
        
        self.block2_0 = IdentityBlock(64,128,kernel_size =1 ,stride =2)                  
        self.block2_1 = ResidualBlock(64,128,stride1=2,stride2=1)                                                
        
        self.block3_0 = ResidualBlock(128,128,stride1=1,stride2=1)
        self.block3_1 = ResidualBlock(128,128,stride1=1,stride2=1)
        self.block3_2 = ResidualBlock(128,128,stride1=1,stride2=1)
        
        self.block4_0 = IdentityBlock(128,256,kernel_size =1 ,stride =2)
        self.block4_1 = ResidualBlock(128,256,stride1=2,stride2=1)

        self.block5_0 = ResidualBlock(256,256,stride1=1,stride2=1)
        self.block5_1 = ResidualBlock(256,256,stride1=1,stride2=1)
        self.block5_2 = ResidualBlock(256,256,stride1=1,stride2=1)
        self.block5_3 = ResidualBlock(256,256,stride1=1,stride2=1)
        self.block5_4 = ResidualBlock(256,256,stride1=1,stride2=1)
        
        self.block6_0 = IdentityBlock(256,512,kernel_size =1 ,stride =2)
        self.block6_1 = ResidualBlock(256,512,stride1=2,stride2=1)
        
        self.block7_0 = ResidualBlock(512,512,stride1=1,stride2=1)
        self.block7_1 = ResidualBlock(512,512,stride1=1,stride2=1)
        
        self.fc1 = nn.Linear(512*4*4,512)
        self.fc2 = nn.Linear(512,10)
        
        self.bnorm = nn.BatchNorm1d(512)

        
        
    def forward(self,x):
        
        x = self.block1_0(x)
        
        shortcut = x
        x = self.block1_1(x)
        x = x + shortcut
        
        shortcut = x
        x = self.block1_2(x)
        x = x + shortcut
            
        shortcut = x
        x = self.block1_3(x)
        x = x + shortcut

        
        shortcut = x
        x = self.block2_1(x)
        shortcut = self.block2_0(shortcut) 
        x = x + shortcut

        
        shortcut = x 
        x = self.block3_0(x)
        x = x + shortcut
        
        shortcut = x 
        x = self.block3_1(x)
        x = x + shortcut
        
        shortcut = x 
        x = self.block3_2(x)
        x = x + shortcut
        
        
        
        shortcut = x
        shortcut = self.block4_0(shortcut)
        x = self.block4_1(x)
        x = x + shortcut



        shortcut = x 
        x = self.block5_0(x)
        x = x + shortcut
        
        shortcut = x 
        x = self.block5_1(x)
        x = x + shortcut
        
        shortcut = x 
        x = self.block5_2(x)
        x = x + shortcut
        

        shortcut = x 
        x = self.block5_3(x)
        x = x + shortcut
        
        shortcut = x 
        x = self.block5_4(x)
        x = x + shortcut

        shortcut = x
        shortcut = self.block6_0(shortcut)
        x = self.block6_1(x)
        x = x + shortcut
        
        shortcut = x 
        x = self.block7_0(x)
        x = x + shortcut
        
        shortcut = x 
        x = self.block7_1(x)
        x = x + shortcut
        
        
        x = x.view(x.shape[0],512*4*4)
        
        x = self.fc1(x)
        x = F.relu(self.bnorm(x))
        
        x = self.fc2(x)
        return x

In [18]:
resnet = Resnet32()

In [19]:
device = 'cuda'
resnet.to(device)

Resnet32(
  (block1_0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (block1_1): ResidualBlock(
    (resblock): Sequential(
      (0): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): ReLU()
      (3): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (4): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (5): ReLU()
    )
  )
  (block1_2): ResidualBlock(
    (resblock): Sequential(
      (0): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): ReLU()
      (3): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (4): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (5): ReLU()
    )
  )
  (block1_3): ResidualBlock(
    

In [20]:
train_loader,test_loader = make_batches(128)

/tmp/ipykernel_26/53764489.py:19: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  training_image = torch.tensor(torch.cat((torch.tensor(batch_dict['batch_1'][b'data']),torch.tensor(batch_dict['batch_2'][b'data']),torch.tensor(batch_dict['batch_3'][b'data']),torch.tensor(batch_dict['batch_4'][b'data']),torch.tensor(batch_dict['batch_5'][b'data'])),0),dtype = torch.float32)
/tmp/ipykernel_26/53764489.py:20: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  training_labels = torch.tensor(torch.cat((torch.tensor(batch_dict['batch_1'][b'labels']),torch.tensor(batch_dict['batch_2'][b'labels']),torch.tensor(batch_dict['batch_3'][b'labels']),torch.tensor(batch_dict['batch_4'][b'labels']),torch.tensor(

In [21]:
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.AdamW(resnet.parameters(),lr = 0.001)

In [22]:
new_model =  training(resnet,criterion,optimizer,train_loader,epochs = 15)


epoch:1,loss:1.5379
epoch:1,loss:1.2984
epoch:1,loss:1.1724
accuracy:50.304
epoch:2,loss:0.9863
epoch:2,loss:0.7923
epoch:2,loss:0.6749
accuracy:70.902
epoch:3,loss:0.5341
epoch:3,loss:0.5723
epoch:3,loss:0.6286
accuracy:79.222
epoch:4,loss:0.5458
epoch:4,loss:0.5242
epoch:4,loss:0.5037
accuracy:84.256
epoch:5,loss:0.3389
epoch:5,loss:0.2311
epoch:5,loss:0.4087
accuracy:88.56
epoch:6,loss:0.1586
epoch:6,loss:0.2052
epoch:6,loss:0.2786
accuracy:92.374
epoch:7,loss:0.1078
epoch:7,loss:0.1210
epoch:7,loss:0.1871
accuracy:94.806
epoch:8,loss:0.0634
epoch:8,loss:0.0705
epoch:8,loss:0.1010
accuracy:96.258
epoch:9,loss:0.0490
epoch:9,loss:0.0615
epoch:9,loss:0.0726
accuracy:97.456
epoch:10,loss:0.1197
epoch:10,loss:0.0185
epoch:10,loss:0.1188
accuracy:97.938
epoch:11,loss:0.0695
epoch:11,loss:0.1090
epoch:11,loss:0.0496
accuracy:97.968
epoch:12,loss:0.0122
epoch:12,loss:0.0837
epoch:12,loss:0.0422
accuracy:98.186
epoch:13,loss:0.0258
epoch:13,loss:0.0359
epoch:13,loss:0.0393
accuracy:98.302
e

In [23]:
evaluation(resnet,test_loader)


Accuracy of the network on the 10000 test images: 82.93 %
